# NML download notebook
This notebook downloads data from No More Leaks and import it into the NML database

# Set constants

In [ ]:
# Keep the same
DATA_DIR="../NML"


# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

In [ ]:
import difflib
import subprocess

# Download data

In [ ]:
!rm $DATA_DIR/*.md5

In [ ]:
# Get md5 files
!(cd $DATA_DIR;sftp  -i ../conf/divd_nml_id_ed25519  -b ../5.no_more_leaks/nml_get_md5.txt divd@dl.nomoreleaks.org)

In [ ]:
# Cleaning batch file
!rm $DATA_DIR/*.txt
!cp nml_get_template.txt $DATA_DIR/nml_get_files.txt

In [ ]:
md5_files = glob(f"{DATA_DIR}/*.md5")
fh_out = open(f"{DATA_DIR}/nml_get_files.txt", "a")
# Fix checksums
for md5_file in md5_files:
    md5 = open(md5_file, "r").readlines()
    if len(md5[0]) == 32 :
        with open(md5_file, "w") as fh:
            print(f"{md5[0]}  {os.path.basename(md5_file[0:-4])}", file=fh, end="")
# Determine if we need to get files
need_to_get = False
for md5_file in md5_files:
    fh1 = open(md5_file, "r")
    getit = False
    if not os.path.exists(f"{DATA_DIR}/LAST/{os.path.basename(md5_file)}") : 
        # If we have an .md5 file but no data file we need to download it
        getit = True
        need_to_get = True
        print(f"New .md5 file {os.path.basename(md5_file)}.md5, adding {os.path.basename(md5_file)} to the download list")
    if not getit :
        # If the .md5 file we got is new of different from the one we got the last file, we need to download the data file
        fh2 = open(f"{DATA_DIR}/LAST/{os.path.basename(md5_file)}")
        md51 = fh1.readlines()
        md52 = fh2.readlines()
        diff = difflib.unified_diff(md51,md52)
        if len(list(diff)) > 0 :
            getit = True
            print(f"Updated .md5 file {os.path.basename(md5_file)}.md5, adding {os.path.basename(md5_file)} to the download list")
    if not getit :
        print(f"MD5 checking {os.path.basename(md5_file)}....", end="")
        # If the datafile we have does not match the .md5 file we just got, we need to download the file
        check_out = subprocess.check_output(f"cd {DATA_DIR};md5sum --check {os.path.basename(md5_file)}", shell=True).decode("utf-8")
        if check_out[-3:-1] != 'OK' :
            print("NO MATCH")
            get_it = True
            print(f"MD5 in .md5 file {os.path.basename(md5_file)}.md5 doesn;t match data file, adding {os.path.basename(md5_file)} to the download list")
        else:
            print("OK")
    if getit:
        # Add the file to the sftp batch file
        print(f"get {os.path.basename(md5_file)[0:-4]}", file=fh_out)
        need_to_get = True
print("quit", file=fh_out)
fh_out.close()
if not need_to_get:
    print("No need to get any file, you are up to date.")

In [ ]:
if need_to_get:
    !cat $DATA_DIR/*.txt

In [ ]:
# Get data files
if need_to_get:
    !(cd $DATA_DIR;sftp  -i ../conf/divd_nml_id_ed25519  -b nml_get_files.txt divd@dl.nomoreleaks.org)
else:
    print("Not getting anything")

In [ ]:
# CHeck all md5 sums
!(cd $DATA_DIR;md5sum --check *.md5)

In [ ]:
# Copy MD5 files to LAST to confirm we have downloaded the data
!cp $DATA_DIR/*.md5 $DATA_DIR/LAST/